In [ ]:
import vk_api
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.dates as mdates
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
from transformers import pipeline
import string
from nltk import word_tokenize, SnowballStemmer

In [ ]:
!pip install torch torchvision torchaudio

## Загрузка данных

Получим доступ к аккаунту через access_token(его получить можно например здесь https://vkhost.github.io/)

In [ ]:
access_token = 'YOUR_TOKEN_HERE'
vk_session = vk_api.VkApi(token=access_token)
vk = vk_session.get_api()

Сюда вставляем id друга

In [ ]:
friend_id = 999999999

За один запрос мы можем получить не больше 200 сообщений из диалога, так что воспользуемся циклом для запроса всей переписки

In [ ]:
try:
    messages = vk.messages.getHistory(user_id=friend_id, count=200)  
    all_messages = messages['items']
    while messages['count'] > len(all_messages):
        messages = vk.messages.getHistory(user_id=friend_id, count=200, offset=len(all_messages))
        all_messages.extend(messages['items'])
        print(messages['count'], len(all_messages))
except vk_api.VkApiError as error_msg:
    print(error_msg)

Так как по каждому сообщению есть дополнительная информация, то засунем их в датафрейм

In [ ]:
data = pd.DataFrame(all_messages)
data.head(5)

Преобразуем дату в удобный вид

In [ ]:
data['date'] = (data['date']
                       .apply(lambda x: datetime.fromtimestamp(x).strftime('%d.%m.%Y %H:%M:%S')))
data['date'] = pd.to_datetime(data['date'], format='%d.%m.%Y %H:%M:%S')
data = data.set_index('date')
data = data.sort_index(ascending=False)

Для удобства заменим столбец out на наши имена

In [ ]:
YOUR_NAME = 'Kent1'
FRIEND_NAME = 'Kent2'
data['out'] = data['out'].replace({1 : YOUR_NAME, 0 : FRIEND_NAME})

## Анализ количественных характеристик сообщений

Получим датафрейм с нужной информацией

In [ ]:
messages_df = data[['out', 'text']]

In [ ]:
print("Количество сообщений в чате", messages_df.shape[0])
print("Количество сообщений от кажого участника:")
display(messages_df.groupby('out').count());

In [ ]:
print('Дата первого сообщения', messages_df.index[-1])

Рассмотрим количество сообщений за каждый час

In [ ]:
messages_amount_df = messages_df.drop(['text', 'out'], axis=1)
messages_amount_df['count'] = 1
messages_amount_hour_df = messages_amount_df.resample('H').sum()
messages_amount_df = messages_amount_df.resample('D').sum()

In [ ]:
plt.figure(figsize=(20, 10))

plt.plot(messages_amount_df.index, messages_amount_df['count'], label='Значения')
rolling_mean = messages_amount_df['count'].rolling(window=7).mean()
plt.plot(messages_amount_df.index, rolling_mean, label='недельное скользящее среднее', color='red')

plt.title('Количество сообщений по дням')
plt.xlabel('Дата')
plt.ylabel('Количество сообщений')
plt.legend()
plt.show()

In [ ]:
decomposed = seasonal_decompose(messages_amount_df)


plt.figure(figsize=(12, 6))
decomposed.trend.plot()
plt.title('График тренда')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.legend(['Тренд'])
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 6))
result = seasonal_decompose(messages_amount_hour_df)
result.seasonal['2024.03.01':'2024.03.03'].plot()
plt.title('График сезонности по дням')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.legend(['Сезонность'])
plt.grid(True)
plt.show()


plt.figure(figsize=(12, 6))
decomposed.seasonal['2024.03.01':'2024.03.17'].plot()
plt.title('График сезонности по неделям')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.legend(['Сезонность'])
plt.grid(True)
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%a'))
plt.show()


plt.figure(figsize=(12, 6))
decomposed.resid.plot()
plt.title('График остатков')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.legend(['Остатки'])
plt.grid(True)
plt.show()

Посчитаем количество дней в каждом году, где происходило общение

In [ ]:
messages_amount_df['year'] = messages_amount_df.index.year
messages_amount_year_df = (
    messages_amount_df[messages_amount_df['count'] > 0].groupby('year')['count'].count().reset_index())
print('Количество дней общения в каждом году:')
print(messages_amount_year_df)

Посчитаем количество слов в каждом сообщении

In [ ]:
messages_df.loc[:, 'word_count'] = messages_df['text'].apply(lambda x: len(x.split()))

In [ ]:
print('Среднее количество слов в сообщении по диалогу', round(messages_df['word_count'].mean(), 2))
print()
print('Среднее количество слов в сообщении по каждому участнику:')
display(messages_df.groupby('out')['word_count'].mean())

Посчитаем разницу между текущим сообщением и прошлым

In [ ]:
messages_df.loc[:, 'time_diff'] = ((messages_df.index.to_series().shift(1) - messages_df.index)
                                   .fillna(pd.Timedelta(seconds=0)))

Будем считать, что человек начал диалог, если перед его сообщением не было активности в течении N часов

In [ ]:
N = 3
messages_df.loc[:, 'start_conversation'] = (messages_df['time_diff'] > pd.Timedelta(hours=N))
conversation_starts = messages_df.groupby('out')['start_conversation'].sum()
print('Количество начатых диалогов')
print(conversation_starts)

## Анализ содержания сообщений

Добавим в датафрейм лишь сообщение и его отправителя

In [ ]:
messages_text_df = messages_df[['out', 'text']]

Напишем несколько функций для создания облака слов

In [ ]:
# Функция для создания корпуса слов
def str_corpus(corpus):
    str_corpus = ''
    for i in corpus:
        str_corpus += ' ' + i
    str_corpus = str_corpus.strip()
    return str_corpus

# Функция для получения всех слов
def get_corpus(data):
    corpus = []
    for phrase in data:
        for word in phrase.split():
            corpus.append(word)
    return corpus

# Функция для созданий WordCloud
def get_wordcloud(corpus):
    wordCloud = WordCloud(
        background_color='Black',
        width=3000,
        height=2500,
        max_words=200,
        random_state=42
    ).generate(str_corpus(corpus)) 
    return wordCloud

Создадим облако слов

In [ ]:
fig = plt.figure(figsize=(20, 8))
corpus = get_corpus(messages_df['text'].values)
wordcloud = get_wordcloud(corpus)
plt.imshow(wordcloud)
plt.axis('off')

Тут слишком много всяких лишних слов, так что удалим их.

In [ ]:
nltk.download('stopwords')

Загрузим уже готовый список стоп-слов

In [ ]:
stop_words_set = set(stopwords.words('russian'))

Добавим более редкие, встречающиеся только в нашем диалоге

In [ ]:
stop_words_set.update(( 'ахах', 'ахахах', 'хех', 'хах', 'ахахa',  'а', 'ахахаха',
                        'вообще','просто', 'вроде', 'очень', 'тип', 'где то', 'мб', 'https', 'это', 'ещё',
                        'думаю','всё', 'ага', 'понял', 'пока', 'спасибо', 'точно', 'норм', 'такое', 'че',
                        ' ахаха '))

Перепишем функцию создания облака, для использования фильтра

In [ ]:
def get_wordcloud(corpus, stop_words_set):
    wordCloud = WordCloud(
        background_color='black',
        width=3000,
        height=2500,
        max_words=200,
        stopwords=stop_words_set,
        random_state=42
    ).generate(str_corpus(corpus)) 
    return wordCloud

In [ ]:
fig = plt.figure(figsize=(20, 8))
corpus = get_corpus(messages_df['text'].values)
wordcloud_without_stop = get_wordcloud(corpus,stop_words_set)
plt.imshow(wordcloud_without_stop)
plt.axis('off')

In [ ]:
fig = plt.figure(figsize=(20, 8))

corpus_friend = get_corpus(messages_df[messages_df['out'] == FRIEND_NAME]['text'].values)
wordcloud_friend = get_wordcloud(corpus_friend, stop_words_set)
plt.subplot(1, 2, 1)
plt.imshow(wordcloud_friend)
plt.title(FRIEND_NAME)
plt.axis('off')

corpus_me = get_corpus(messages_df[messages_df['out'] == YOUR_NAME]['text'].values)
wordcloud_me = get_wordcloud(corpus_me, stop_words_set)
plt.subplot(1, 2, 2)
plt.imshow(wordcloud_me)
plt.title(YOUR_NAME)
plt.axis('off')
plt.show();

Проделаем тоже самое, предварительно проведя стеммизацию

In [ ]:
def process_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text.lower())
    stop_words_set = set(stopwords.words('russian'))
    stop_words_set.update(('ахах', 'ахахах', 'хех', 'хах', 'ахахa', 'а', 'ахахаха',
                       'вообще', 'просто', 'вроде', 'очень', 'тип', 'где то', 'мб', 'https', 'это', 'ещё',
                       'думаю', 'всё', 'ага', 'понял', 'пока', 'спасибо', 'точно', 'норм', 'такое', 'че',
                       ' ахаха '))

    stemmer = SnowballStemmer('russian')
    text = [stemmer.stem(word) for word in tokens if word not in stop_words_set]
    return ' '.join(text)

In [ ]:
messages_df['text_stem'] = messages_df['text'].apply(process_text)

In [ ]:
fig = plt.figure(figsize=(20, 8))

corpus_friend = get_corpus(messages_df[messages_df['out'] == FRIEND_NAME]['text_stem'].values)
wordcloud_friend = get_wordcloud(corpus_friend, stop_words_set)
plt.subplot(1, 2, 1)
plt.imshow(wordcloud_friend)
plt.title(FRIEND_NAME)
plt.axis('off')

corpus_me = get_corpus(messages_df[messages_df['out'] == YOUR_NAME]['text_stem'].values)
wordcloud_me = get_wordcloud(corpus_me, stop_words_set)
plt.subplot(1, 2, 2)
plt.imshow(wordcloud_me)
plt.title(YOUR_NAME)
plt.axis('off')
plt.show();

Проведём sentiment analysis, чтобы узнать кто тут токсик, загрузим уже обученную модель. Понять, что выкупать локальные мемы она не будет, но общую оценку мы сможем увидеть

In [ ]:
classifier = pipeline("sentiment-analysis", model="seara/rubert-tiny2-russian-sentiment")
def analyze_sentiment_bert(text):
    result = classifier(text)[0]
    label = result['label']
    score = result['score']
    return label, score

Обрежем слишком длинные сообщения

In [ ]:
MAX_TEXT_LENGTH = 512  
def analyze_sentiment_bert_wrapper(text):
    text = text[:MAX_TEXT_LENGTH]
    return analyze_sentiment_bert(text)

In [ ]:
messages_text_df[['sentiment_label', 'sentiment_score']] = (
    messages_text_df['text'].apply(analyze_sentiment_bert_wrapper).apply(pd.Series))

In [ ]:
messages_text_df.groupby(['out', 'sentiment_label'])['text'].count()

Посмотрим теперь на эти сообщения

In [ ]:
(messages_text_df[messages_text_df['sentiment_label'] == 'positive']
 .sort_values('sentiment_score', ascending=False)).head(10)

In [ ]:
(messages_text_df[messages_text_df['sentiment_label'] == 'negative']
 .sort_values('sentiment_score', ascending=False)).head(10)